# Setup do Projeto

In [5]:
# Testando CUDA
import torch
print(f"CUDA está disponível: {torch.cuda.is_available()}\nDevice count: {torch.cuda.device_count()}\nNome do dispositivo: {torch.cuda.get_device_name(0)}\nVersao do PyTorch: {torch.__version__}")

CUDA está disponível: True
Device count: 2
Nome do dispositivo: AMD Radeon RX 7800 XT
Versao do PyTorch: 2.9.1+rocm7.2.0.git7e1940d4


In [6]:
# Setup Agnostic Code for GPU/CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
# Constantes
DATA_PATH = '/home/jose-vitor/Documents/Cityscapes_Dataset'
BATCH_SIZE = 64

# Importar o Dataset Cityscapes

In [91]:
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

Definir Lables

In [63]:
# Importar tabela de lables
labels_df = pd.read_csv('labels_cityscapes.csv')

# Excluir lablel -1 (classe não usada)
# Substitur trainID de 255 para 19 para facilitar a conversao
labels_df = labels_df[labels_df['trainID'] != -1].replace({'trainID': {255: 19}})

# Criar novo dataframe com labels usados para treino
train_labels_df = labels_df.loc[(labels_df['trainID'] != 255) & (labels_df['trainID'] != -1)]
train_labels_df = train_labels_df.drop(columns=['id', 'trainID'])
train_labels_df.loc[-1] = ['ignore', 'ignore', 255, False, True, (0, 0, 0)]  # adiciona linha para classe ignore, que deve ser desconsiderada no treino
train_labels_df = train_labels_df.reset_index(drop=True) # reseta os indexes
train_labels_df

,name,category,catID,hasinstances,ignoreInEval,color
0,unlabeled,void,0,False,True,"(0, 0, 0)"
1,ego vehicle,void,0,False,True,"(0, 0, 0)"
2,rectification border,void,0,False,True,"(0, 0, 0)"
3,out of roi,void,0,False,True,"(0, 0, 0)"
4,static,void,0,False,True,"(0, 0, 0)"
5,dynamic,void,0,False,True,"(111, 74, 0)"
6,ground,void,0,False,True,"(81, 0, 81)"
7,road,flat,1,False,False,"(128, 64, 128)"
8,sidewalk,flat,1,False,False,"(244, 35, 232)"
9,parking,flat,1,False,True,"(250, 170, 160)"


In [64]:
# Definindo dicionario para conversao de ids para treino
id_to_trainid = dict(zip(labels_df['id'], labels_df['trainID']))
id_to_trainid

{0: 19,
 1: 19,
 2: 19,
 3: 19,
 4: 19,
 5: 19,
 6: 19,
 7: 0,
 8: 1,
 9: 19,
 10: 19,
 11: 2,
 12: 3,
 13: 4,
 14: 19,
 15: 19,
 16: 19,
 17: 5,
 18: 19,
 19: 6,
 20: 7,
 21: 8,
 22: 9,
 23: 10,
 24: 11,
 25: 12,
 26: 13,
 27: 14,
 28: 15,
 29: 19,
 30: 19,
 31: 16,
 32: 17,
 33: 18}

In [92]:
# Criando o cocour map para imprimir imagens segmentadas
train_color_map = train_labels_df['color'].to_list()
train_color_map = ['#%02x%02x%02x' % color for color in train_color_map]
train_cmap = mcolors.LinearSegmentedColormap.from_list('CityscapesTrainColorMap', train_color_map, N=len(train_color_map))

TypeError: %x format: an integer is required, not str

Definir Funções de Transform

In [65]:
# Funcoes personalizadas para transformacoes

# -- 1. remove canal extra desnecessário na segmentação
lable_squeeze = lambda x: x.squeeze(dim=0)

# -- 2. transforma os ids originais para ids de treino
class IdToTrainIdTransform:

    def __init__(self, conv_dict: dict):
        self.conv_dict = conv_dict

    def __call__(self, lable: torch.Tensor) -> torch.Tensor:
        new_lable = lable # cria uma copia da lable original para ser modificada
        # Troca os valores da lable original a partir do dicionario
        for key, value in list(self.conv_dict.items()):
            # Caso o pixel esteja com a lable da key, troca para a lable do value, caso controario mantem a lable original
            new_lable = torch.where(new_lable == key, value, new_lable)
        return new_lable
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}()"

In [66]:
# transformando dados em tensores e aplicando data augmentation
# O dataset original possui tamanho 1024x2048

# -- DATA TRANSFORMS -- #
train_transform = v2.Compose([
    v2.Resize(size=(256,512), interpolation=InterpolationMode.BILINEAR), # redimensiona imagem para 256x512
    v2.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET), # aplica data augmentation
    v2.PILToTensor(), # converte imagem PIL para tensor
    v2.ToDtype(torch.float32, scale=True) # normaliza para [0,1]
])

val_transform = v2.Compose([
    v2.Resize(size=(256,512)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True)
])

# -- TARGET TRANFROMS -- #
target_transform = v2.Compose([
    v2.PILToTensor(), # converte segmentação PIL para tensor
    IdToTrainIdTransform(id_to_trainid), # converte ids originais para ids de treino
    v2.Resize(size=(256,512), interpolation=InterpolationMode.NEAREST_EXACT), # redimensiona imagem para 256x512. Nearest Neighbor para nao criar novos valores
    v2.Lambda(lable_squeeze), # remove canal extra desnecessário na segmentação
    v2.ToDtype(torch.uint8) # apenas converte para inteiro sem normalizacao
])

Importar Dataset

In [68]:
train_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'train',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=train_transform,
                                    target_transform=target_transform)
val_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'val',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=val_transform,
                                    target_transform=target_transform)
test_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'test',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=val_transform,
                                    target_transform=target_transform)


In [109]:
# Imprimir informacoes importantes dos datasets
img, smnt = train_dataset[0]
print(f"O Dataset de treino possui {len(train_dataset)} amostras.\n"
      f"O Dataset de validação possui {len(val_dataset)} amostras.\n"
      f"O Dataset de teste possui {len(test_dataset)} amostras.\n"
      f"Cada imagem posssui tamanho {img.shape} e tipo {img.dtype}, e cada lable possui tamanho {smnt.shape} e tipo {smnt.dtype}.")

O Dataset de treino possui 2975 amostras.
O Dataset de validação possui 500 amostras.
O Dataset de teste possui 1525 amostras.
Cada imagem posssui tamanho torch.Size([3, 256, 512]) e tipo torch.float32, e cada lable possui tamanho torch.Size([256, 512]) e tipo torch.uint8.


In [ ]:
# Imprimindo imagens e segmentações do dataset
def img_show(imgs: list[torch.Tensor], smnts: list[torch.Tensor] = None, n: int=5):
    plt.figure(figsize=(15,10))
    for i in range(n):
        plt.subplot(n, 2, 2*i+1)
        plt.imshow(imgs[i].permute[1,2,0]) # permute para mudar a ordem dos canais e converter um tensor para imagem
        

ValueError: too many values to unpack (expected 2)

Criar dataloaders